# Ray Basics


![](ray_header_logo.png)



Ray is a growing project not yet at v1.0, its growing fast, is supported by a commerical outfit AnyScale

[The documentation is great but big](https://docs.ray.io/en/latest/index.html)

[There are great tutorials](https://docs.ray.io/en/latest/auto_examples/overview.html#ray-tutorials-and-examples)

For tune there are a set of [colab notebooks with cool examples](https://docs.ray.io/en/latest/tune/tutorials/overview.html)

Here we are just going to look at one way of using Ray for some in domain examples


In [1]:
%load_ext autoreload
%autoreload 2

from dependencies import *

Loading dependencies we have already seen...
Done...


In [2]:
import ray

Start the ray "cluster" and go and check the dashboard

 - cpus
 - gpus
 - local_mode

In [3]:
ray.init(num_cpus=10, include_dashboard=True,
         dashboard_host="localhost")

2020-11-09 11:07:01,768	INFO services.py:1164 -- View the Ray dashboard at http://localhost:8266


{'node_ip_address': '192.168.123.68',
 'raylet_ip_address': '192.168.123.68',
 'redis_address': '192.168.123.68:12760',
 'object_store_address': '/tmp/ray/session_2020-11-09_11-07-01_050983_11592/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-09_11-07-01_050983_11592/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-11-09_11-07-01_050983_11592',
 'metrics_export_port': 61278}

In [22]:
ray.shutdown()

Ray allows you to define remote functions that will be executed on the cluster

In [4]:
@ray.remote # this is a decorator
def f(x):
    x = 0
    for burn_cycles in range(20000): # burn more cysles to see the cluster blip
        x += burn_cycles
    return x**0.5

In [5]:
futures = [f.remote(i) for i in range(400)]
print(ray.get(futures))
# a future is created as soon as the job runs,

[14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206592083, 14141.78206

Ray allows you to write object based code which they call `Actors` which can maintian state, and also be executed on the cluster

In [26]:
ray.shutdown()
ray.init(num_cpus=10)

2020-11-06 15:11:59,611	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.2.140',
 'raylet_ip_address': '192.168.2.140',
 'redis_address': '192.168.2.140:6379',
 'object_store_address': '/tmp/ray/session_2020-11-06_15-11-59_196677_4568/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-06_15-11-59_196677_4568/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2020-11-06_15-11-59_196677_4568',
 'metrics_export_port': 65099}

In [6]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1

    def read(self):
        return self.n

In [7]:
# setup N actors
counters = [Counter.remote() for i in range(20)]

# call actions on the Actors
[c.increment.remote() for c in counters]

# retrieve data from the actors
futures = [c.read.remote() for c in counters]
print(ray.get(futures))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
ray.shutdown()